# Feature Extraction - Wav2Vec2 Embeddings

Loading audio files, pass them through pretrained Wav2Vec2 model. 
Save the embeddings for classification.

In [3]:
# check, listing saved embeddings
import os

# folder containing processed embeddings
folder = "deepfake_audio_project/data/processed_embeddings/"

# list files - confirm feature extraction
print(os.listdir(folder))


['file1778.pt.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file5046.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file22542.pt.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file9670.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file17478.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file27687.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file3928.pt.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file24357.pt.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file14054.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file18421.pt.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file10992.pt.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file5288.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file7812.pt.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file20310.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file22908.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file56.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', 'file28734.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt', '

In [1]:
# install dependencies
!pip install transformers torchaudio tqdm

# imports
import os
import torch
import torchaudio
from transformers import Wav2Vec2Model, Wav2Vec2Processor
from tqdm import tqdm
import time

# setting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using device: {device}")

# loading the pretrained Wav2Vec2 model 
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)
model.eval() # evaluation mode 

# paths - where raw audio files are and where embeddings will go 
DATA_DIR = "deepfake_audio_project/data/raw/for-2sec/for-2seconds"
SAVE_DIR = "deepfake_audio_project/data/processed_embeddings"


# skip logic: list already processed, skip if file exists! 
already_processed = set(os.listdir(SAVE_DIR))

# collect all audio files needing processing
audio_paths = []

for root, dirs, files in os.walk(DATA_DIR):
    for file in files:
        if file.endswith('.wav') or file.endswith('.mp3'):
            full_path = os.path.join(root, file)
            
            # infer label from parent folder name
            label = os.path.basename(os.path.dirname(full_path))
            
            # infer split (training/validation/testing) from grandparent folder
            split = os.path.basename(os.path.dirname(os.path.dirname(full_path)))

            # skip if processed
            save_name = file.replace(".wav", ".pt").replace(".mp3", ".pt")
            if save_name not in already_processed:
                audio_paths.append((full_path, label, split))

total_files = len(audio_paths)
print(f"found {total_files} files to process (skipping already done).")

# extracting embeddings now! 

# timer - for tracking purposes
start_time = time.time()

# feature extraction loop with tqdm
for idx in tqdm(range(total_files), desc="Extracting features"):
    path, label, split = audio_paths[idx]   # unpack full path, label and split
    
    try:
        # load audio file
        speech_array, sampling_rate = torchaudio.load(path)
        speech_array = speech_array.squeeze(0) # remove channel dimension if it exists 

        # resample if needed to 16kHz 
        if sampling_rate != 16000:
            resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
            speech_array = resampler(speech_array)

        # preprocess for wav2vec2 input 
        input_values = processor(speech_array.numpy(), sampling_rate=16000, return_tensors="pt").input_values.to(device)

        # extract features with wav2vec2
        with torch.no_grad(): # no gradient tracking needed 
            embeddings = model(input_values).last_hidden_state.mean(dim=1).squeeze(0)
        
        # save extracted embedding, label, and split 
        save_name = os.path.basename(path).replace(".wav", ".pt").replace(".mp3", ".pt")
        torch.save({
            "embedding": embeddings.cpu(),
            "label": label,
            "split": split  
        }, os.path.join(SAVE_DIR, save_name))
    
    except Exception as e:
        print(f"error processing {path}: {e}")

    # live progress print every 100 files
    if (idx + 1) % 100 == 0 or (idx + 1) == total_files:
        elapsed = time.time() - start_time
        print(f"processed {idx + 1}/{total_files} files [{(idx + 1) / total_files * 100:.2f}%] - Elapsed {elapsed/60:.1f} min")

print("\n done extracting and saving all embeddings!")


  Using cached torchaudio-2.6.0-cp39-cp39-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached torch-2.6.0-cp39-cp39-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata 

/stor/home/spl742/wavenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using device: cuda


Using the `SDPA` attention implementation on multi-gpu setup with ROCM may lead to performance issues due to the FA backend. Disabling it to use alternative backends.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


found 17870 files to process (skipping already done).


...

processed 17700/17870 files [99.05%] - Elapsed 5.5 min


Extracting features: 100%|█████████▉| 17812/17870 [05:33<00:00, 81.87it/s]

processed 17800/17870 files [99.61%] - Elapsed 5.6 min


Extracting features: 100%|██████████| 17870/17870 [05:34<00:00, 53.44it/s]

processed 17870/17870 files [100.00%] - Elapsed 5.6 min

 done extracting and saving all embeddings!


In [4]:
# example of embedding file 

# sample file to make sure it contains all elements 
example = torch.load("deepfake_audio_project/data/processed_embeddings/file1778.pt.pt_16k.pt_norm.pt_mono.pt_silence.pt_2sec.pt")
print(example.keys())


dict_keys(['embedding', 'label', 'split'])
